In [2]:
'''
In this project, we will perform comprehensive data cleaning and analysis on an e-commerce dataset to prepare it for further exploration and business intelligence. The dataset, obtained from Kaggle, includes transaction records from an online retail store. The objective is to clean the data, remove inconsistencies, and transform it into a format suitable for analysis.

Objectives:
Data Import and Exploration:

Import the dataset from CSV format into a SQL-based database.
Explore the dataset to understand its structure and identify potential issues.
Data Cleaning:

Remove Rows with Missing Values: Delete records with any missing critical fields, including InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, and Country.
Remove Duplicate Records: Identify and remove any duplicate transactions to ensure data accuracy.
Correct Data Types: Ensure that fields like Quantity, UnitPrice, and CustomerID are of the correct numeric types and InvoiceDate is formatted as a date.
Data Transformation:

Create Derived Columns: Add new columns to calculate total revenue per transaction.
Parse Dates: Extract and create new columns for InvoiceYear and InvoiceMonth from the InvoiceDate.
Categorize Prices: Create a PriceCategory column based on unit price ranges.
Handling Special Cases:

Identify and Handle Returns: Separate negative quantity transactions (returns) into a different table for specialized analysis.
Detect Anomalies: Identify unusual patterns such as high transaction amounts or frequent purchases by the same customer.
Data Aggregation:

Monthly Revenue Analysis: Aggregate data to calculate total revenue per month and create summary tables for business reporting.
Customer Purchase Patterns: Analyze purchase behavior by customer, including transaction frequency and total expenditure.
Normalization and Denormalization:

Normalize Data: Separate transactional data from customer data for improved organization and analysis.
Denormalize Data: Combine tables for easier reporting and analysis if necessary.
Performance Optimization:

Indexing: Create indexes on frequently queried columns to improve query performance.
Data Export: Export the cleaned and transformed data for use in further analysis or reporting tools.
Documentation and Reporting:


'''
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sqlalchemy import create_engine

# Input data files are available in the read-only "../input/" directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load dataset using pandas with a different encoding
df = pd.read_csv('/kaggle/input/ecommerce-data/data.csv', encoding='ISO-8859-1')

# Create an in-memory SQLite database
engine = create_engine('sqlite://', echo=False)

# Load dataframe into SQL table
df.to_sql('ecommerce_data', con=engine, index=False, if_exists='replace')

# Selecting the first 5 rows from the table
query = "SELECT * FROM ecommerce_data LIMIT 5;"
result = pd.read_sql_query(query, engine)
print(result)  # Print the first 5 rows of the table

# SQL Queries for data cleaning and transformation
queries = [
    '''
    -- Query 1: Delete rows with NULL values in important columns.
    DELETE FROM ecommerce_data 
    WHERE InvoiceNo IS NULL 
    OR StockCode IS NULL
    OR Description IS NULL
    OR Quantity IS NULL
    OR InvoiceDate IS NULL
    OR UnitPrice IS NULL
    OR CustomerID IS NULL
    OR Country IS NULL;
    ''',

    '''
    -- Query 2: Remove duplicate rows.
    DELETE FROM ecommerce_data
    WHERE rowid NOT IN (
        SELECT MIN(rowid)
        FROM ecommerce_data
        GROUP BY InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country
    );
    ''',

    '''
    -- Query 3: Standardize the format of the InvoiceDate column.
    UPDATE ecommerce_data
    SET InvoiceDate = strftime('%Y-%m-%d %H:%M:%S', InvoiceDate);
    ''',

    '''
    -- Query 4: Delete rows with non-positive quantities.
    DELETE FROM ecommerce_data WHERE Quantity <= 0;
    ''',

    '''
    -- Query 5: Remove rows with invalid UnitPrice values.
    DELETE FROM ecommerce_data WHERE UnitPrice <= 0 OR UnitPrice > 1000;
    ''',

    '''
    -- Query 6: Standardize the format of Country and Description.
    UPDATE ecommerce_data
    SET Country = UPPER(Country), Description = UPPER(Description);
    ''',

    '''
    -- Query 7: Remove rows with invalid CustomerID values.
    DELETE FROM ecommerce_data WHERE CustomerID IS NULL OR CustomerID < 0;
    ''',

    '''
    -- Query 8: Add and calculate a new TotalRevenue column.
    ALTER TABLE ecommerce_data ADD COLUMN TotalRevenue DECIMAL(10, 2);
    UPDATE ecommerce_data
    SET TotalRevenue = Quantity * UnitPrice;
    ''',

    '''
    -- Query 9: Add a PriceCategory column based on UnitPrice values.
    ALTER TABLE ecommerce_data ADD COLUMN PriceCategory VARCHAR(50);
    UPDATE ecommerce_data
    SET PriceCategory = CASE
        WHEN UnitPrice < 1 THEN 'Low'
        WHEN UnitPrice BETWEEN 1 AND 10 THEN 'Medium'
        ELSE 'High'
    END;
    ''',

    '''
    -- Query 10: Log the data cleaning operations.
    CREATE TABLE IF NOT EXISTS data_cleaning_log (
        ChangeDate TIMESTAMP,
        ChangeDescription TEXT
    );
    INSERT INTO data_cleaning_log (ChangeDate, ChangeDescription)
    VALUES (CURRENT_TIMESTAMP, 'Data cleaning operations completed.');
    '''
]

# Execute the queries
try:
    with engine.connect() as connection:
        # Begin a transaction
        with connection.begin():
            for query in queries:
                connection.execute(query)
    print("Data cleaning operations completed successfully.")
except Exception as e:
    print(f"An error occurred while executing queries: {e}")

# Validate the result after cleaning
try:
    cleaned_result = pd.read_sql_query(query, engine)
    print("First 5 rows after cleaning:")
    print(cleaned_result)
except Exception as e:
    print(f"An error occurred while fetching data: {e}")


/kaggle/input/ecommerce-data/data.csv
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  
An error occurred while executing queries: Not an executable object: '\n    -- Query 1: Delete rows with NULL values in important columns.\n    DELETE FROM ecommerce_data \n  